# IFC Rasterization for Efficient Spatial Analysis

## Intro

In [19]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
from importlib import reload 
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
import vtk
from concurrent.futures import ThreadPoolExecutor

def to_vtk_faces(faces : Tuple[tuple]) -> np.ndarray:

    faces=np.array(faces, dtype=np.int16)
    num_insertions = (len(faces) - 1) // 3

    # Generate an array of indices for insertions
    indices = np.arange(3, 3 * (num_insertions + 1), 3)
    indices = np.insert(indices, 0, 0)
    faces = np.insert(faces, indices, 3) 
    return faces

def vtk_block_by_building_element(ifc_file):
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    exclude_list = ["IfcSpace", "IfcOpeningElement", "IfcWindow", "IfcDoor"]
    
    multiblock = pv.MultiBlock()

    if iterator.initialize():

        while True:
            shape = iterator.get()
            if shape.type not in exclude_list:
                faces = shape.geometry.faces
                verts = shape.geometry.verts
                poly_data = pv.PolyData(list(verts), to_vtk_faces(faces))
                multiblock.append(poly_data)

            if not iterator.next():
                break
               
    return multiblock

class OctreeNode:
    def __init__(self, bounds, level=0):
        self.bounds = bounds
        self.level = level
        self.children = []
        self.is_leaf = True
        self.contains_mesh = False

class Octree:
    def __init__(self, bounds, max_levels):
        self.root = OctreeNode(bounds, level=0)
        self.max_levels = max_levels

def object_fully_contained_in_node(node, mesh):
    node_bounds = node.bounds
    mesh_bounds = mesh.bounds
    return all(node_bounds[i] <= mesh_bounds[i] <= node_bounds[i+1] for i in range(0, 6, 2))

def object_intersects_node(node, mesh):
    node_bounds = node.bounds
    mesh_bounds = mesh.bounds
    return all(node_bounds[i] <= mesh_bounds[i+1] and node_bounds[i+1] >= mesh_bounds[i] for i in range(0, 6, 2))

def subdivide_node(node):
    bounds = node.bounds
    level = node.level
    x_mid = (bounds[0] + bounds[1]) / 2
    y_mid = (bounds[2] + bounds[3]) / 2
    z_mid = (bounds[4] + bounds[5]) / 2

    new_bounds = [
        [bounds[0], x_mid, bounds[2], y_mid, bounds[4], z_mid],
        [x_mid, bounds[1], bounds[2], y_mid, bounds[4], z_mid],
        [bounds[0], x_mid, y_mid, bounds[3], bounds[4], z_mid],
        [x_mid, bounds[1], y_mid, bounds[3], bounds[4], z_mid],
        [bounds[0], x_mid, bounds[2], y_mid, z_mid, bounds[5]],
        [x_mid, bounds[1], bounds[2], y_mid, z_mid, bounds[5]],
        [bounds[0], x_mid, y_mid, bounds[3], z_mid, bounds[5]],
        [x_mid, bounds[1], y_mid, bounds[3], z_mid, bounds[5]],
    ]

    node.children = [OctreeNode(b, level=level+1) for b in new_bounds]
    node.is_leaf = False

def insert_mesh(node, mesh, max_levels):
    if node.level == max_levels or object_fully_contained_in_node(node, mesh):
        node.contains_mesh = True
    else:
        if node.is_leaf:
            subdivide_node(node)
        for child in node.children:
            if object_intersects_node(child, mesh):
                insert_mesh(child, mesh, max_levels)

def get_neighbors(voxel_grid, current_voxel):
    # Implement this function to get the neighbors of the current voxel in the voxel grid
    neighbors = []
    x, y, z = current_voxel
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            for dz in [-1, 0, 1]:
                if dx == dy == dz == 0:
                    continue
                if 0 <= x+dx < voxel_grid.shape[0] and 0 <= y+dy < voxel_grid.shape[1] and 0 <= z+dz < voxel_grid.shape[2]:
                    neighbors.append((x+dx, y+dy, z+dz))
    return neighbors

def recursive_traversal(voxel_grid, current_voxel, visited):
    stack = [current_voxel]
    while stack:
        current_voxel = stack.pop()
        if visited[current_voxel]:
            continue
        visited[current_voxel] = True
        stack.extend(get_neighbors(voxel_grid, current_voxel))

    for neighbor_voxel in get_neighbors(voxel_grid, current_voxel):
        recursive_traversal(voxel_grid, neighbor_voxel, visited)

def get_seed_point(voxel_grid):
    # Implement this function to get a seed point outside the boundary
    # Here, I am assuming that the point (0, 0, 0) is outside the boundary
    return (0, 0, 0)

def fill_volume(voxel_grid, mask):
    interior_voxels = np.logical_not(mask)

    # Get the shape of the voxel_grid
    grid_shape = voxel_grid.shape

    # Get the 1D indices of the interior voxels
    interior_voxel_indices = np.nonzero(interior_voxels)[0]

    # Convert the 1D indices to 3D indices
    interior_voxel_indices_3d = np.array(np.unravel_index(interior_voxel_indices, grid_shape)).T

    # Fill the interior voxels with appropriate values in the 3D voxel grid
    for idx in interior_voxel_indices_3d:
        voxel_grid[tuple(idx)] = 1  # Replace with the actual value you want to use for filling

    return voxel_grid


def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def voxelize_space(meshes, voxel_size, bounds):
    
    """Voxelize space and check intersections with given mesh."""

    grid = create_uniform_grid(bounds, voxel_size)
    voxel_grid = np.zeros((len(grid.x)-1, len(grid.y)-1, len(grid.z)-1), dtype=int)  # Adjusted shape to match cell centers

    num_points = grid.cell_centers().n_points

    mask = np.zeros(num_points, dtype=bool)

    print(f'Total number of voxels: {num_points}')

    select_enclosed = vtk.vtkSelectEnclosedPoints()
    select_enclosed.SetInputData(grid.cell_centers())

    for mesh in meshes:
        select_enclosed.SetSurfaceData(mesh)
        select_enclosed.Update()
        mask = np.logical_or(mask, np.array([select_enclosed.IsInside(i) for i in range(num_points)]))
        
    # Fill the volume inside the voxelized boundary
    voxel_grid = fill_volume(voxel_grid, mask)

    return grid, mask


def voxelize(node, voxel_size, voxelized_representation, meshes):
    if node.is_leaf:
        if node.contains_mesh:
            # Perform voxelization in this region using voxelize_space function
            grid, mask = voxelize_space(meshes, voxel_size, node.bounds)
            voxelized_representation.append((grid, mask))
    else:
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(voxelize, child, voxel_size, meshes) for child in node.children]
            for future in futures:
                voxelized_representation.append(future.result())



## Importing IFC objects

In [20]:
#ifc_file = ifcopenshell.open(r"IFC Files\Project1.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Duplex.ifc")
ifc_file = ifcopenshell.open(r"IFC Files\Clinic.ifc")

# convert IFC objects into vtk meshes
start_time = time.time()

all_meshes = vtk_block_by_building_element(ifc_file)

end_time = time.time()
print(f"Time taken to convert objects: {end_time - start_time:.4f} seconds")

Time taken to convert objects: 27.9845 seconds


## Voxelization

In [21]:
start_time = time.time()

# Create the octree
bounds = all_meshes.bounds # You'll need to determine these based on your data
max_levels = 8 # Adjust based on your needs
octree = Octree(bounds, max_levels)

#merged_polydata = reduce(lambda x, y: x + y, all_meshes)
for mesh in all_meshes:
    insert_mesh(octree.root, mesh, octree.max_levels)

# Perform the voxelization
voxel_size = 0.5  # Adjust based on your needs
voxelized_representation = []
voxelize(octree.root, voxel_size, voxelized_representation, all_meshes)

end_time = time.time()
print(f"Time taken to voxelize: {end_time - start_time:.4f} seconds")

# Visualization
p = pv.Plotter()

for grid, mask in voxelized_representation:
    p.add_mesh(grid.extract_cells(np.where(mask)[0]), color="blue", show_edges=True, opacity=0.5)
p.add_mesh(grid,show_edges=True, opacity=0.3)

p.show()


Total number of voxels: 422940
Time taken to voxelize: 261.5129 seconds


Widget(value="<iframe src='http://localhost:59575/index.html?ui=P_0x11a0e4554e0_1&reconnect=auto' style='width…